In [2]:
pip install mlxtend

     ---------------------------------------- 1.4/1.4 MB 13.2 MB/s eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [29]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

In [46]:
data = pd.read_csv('beginner_recipe.csv')
data = pd.get_dummies(data, columns=['국가 분류'])
columns_to_drop = ['방법 분류', '테마 분류', '난이도 분류', '조회수', '분량', '레시피', '조리시간']
data = data.drop(columns=columns_to_drop)
data['칼로리'] = data['칼로리'].apply(lambda x: 1 if x < 300 else 0)
data.rename(columns={'칼로리': '다이어트'}, inplace=True)

In [47]:
def calculate_spiciness(row):
    score = 0
    spicy_ingredients = ['김치', '고추', '고춧']
    
    # 주재료 이름에서 맵기 성분 찾기
    if isinstance(row['주재료 이름'], str):
        row['주재료 이름'] = eval(row['주재료 이름'])
    
    if isinstance(row['주재료 이름'], list):
        for ingredient in row['주재료 이름']:
            if any(spicy in ingredient for spicy in spicy_ingredients):
                score += 3
                
    # 부재료 이름에서 맵기 성분 찾기
    if isinstance(row['부재료 이름'], str):
        row['부재료 이름'] = eval(row['부재료 이름'])
        
    if isinstance(row['부재료 이름'], list):
        for ingredient in row['부재료 이름']:
            if any(spicy in ingredient for spicy in spicy_ingredients):
                score += 2
    
    # 양념에서 맵기 성분 찾기
    if isinstance(row['양념'], str):
        row['양념'] = eval(row['양념'])
    
    if isinstance(row['양념'], list):
        for ingredient in row['양념']:
            if any(spicy in ingredient for spicy in spicy_ingredients):
                score += 1
                
    return score

data['맵기'] = data.apply(calculate_spiciness, axis=1)
data

,메뉴 이름,다이어트,주재료 이름,부재료 이름,양념,국가 분류_동남아시아,국가 분류_멕시코,국가 분류_서양,국가 분류_이탈리아,국가 분류_일본,국가 분류_중국,국가 분류_퓨전,국가 분류_한식,맵기
0,견과류 영양솥밥,0,"['백미 1과3/4컵', '은행 12개', '밤 8개', '대추 4개', '잣 1큰술']",['물 1과3/4컵'],NaN,0,0,0,0,0,0,0,1,0
1,굴밥,0,"['굴 300g', '쌀 200g', '보리쌀 80g']","['어묵(찐것) 50g', '표고버섯 3장', '당근 50g', '완두콩(통조림) ...",['소금 1작은술'],0,0,0,0,0,0,0,1,0
2,김치볶음밥,0,"['김치(잘게썬것) 1/3포기', '밥 2공기']","['감자 1/4개', '양파 1/2개', '햄 60g', '식용유 약간', '계란 ...","['소금 약간', '후춧가루 약간']",0,0,0,0,0,0,0,1,3
3,꽃상추쌈,0,"['상추 4장', '밥 1그릇']","['다진쇠고기 100g', '다진양파 1/2개']","['간장 1큰술', '설탕 1/2큰술', '다진파 약간', '다진마늘 약간', '깨...",0,0,0,0,0,0,0,1,1
4,녹두죽,1,"['녹두 1컵', '쌀 2컵']",['물 14컵'],"['소금 약간', '설탕 약간']",0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,삶은무해산물볶음,1,"['새우살 50g', '조갯살 50g', '오징어 50g', '무 1개']","['양파 1/4개', '피망 1/4개', '홍고추 1/2개']","['다시용멸치 한줌', '다시마 10 X 10cm 1장', '간장 3큰술', '굴소...",0,0,0,0,0,0,1,0,2
882,엔다이브 위의 쇠고기볶음,0,"['쇠고기 50g', '앤다이브 6잎']","['청피망 1/2개', '홍피망 1/2개', '양파 1/2개']","['올리브오일 1큰술', '굴소스 1큰술', '[양념장] 올리브오일 1큰술', '간...",0,0,0,0,0,0,1,0,0
883,피자떡볶이,1,"['햄 100g', '떡볶이떡 150g']","['어묵 2장', '피망 1/3개', '피자치즈 약간', '양파 1/3개']","['고추장 3큰술', '고춧가루 1작은술', '물엿 1큰술', '꿀 1작은술', '...",0,0,0,0,0,0,1,0,2
884,해물채소볶음,1,"['새우 30g', '바지락 50g', '파프리카 1/2개', '청경채 2개']","['조갯살 30g', '콜리플라워 30g', '가지 1/2개']","['굴소스 3큰술', '참기름 약간', '다진마늘 약간']",0,0,0,0,0,0,1,0,0


In [48]:
columns_to_drop = ['주재료 이름', '부재료 이름', '양념']
CFdata = data.drop(columns=columns_to_drop)
CFdata

,메뉴 이름,다이어트,국가 분류_동남아시아,국가 분류_멕시코,국가 분류_서양,국가 분류_이탈리아,국가 분류_일본,국가 분류_중국,국가 분류_퓨전,국가 분류_한식,맵기
0,견과류 영양솥밥,0,0,0,0,0,0,0,0,1,0
1,굴밥,0,0,0,0,0,0,0,0,1,0
2,김치볶음밥,0,0,0,0,0,0,0,0,1,3
3,꽃상추쌈,0,0,0,0,0,0,0,0,1,1
4,녹두죽,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
881,삶은무해산물볶음,1,0,0,0,0,0,0,1,0,2
882,엔다이브 위의 쇠고기볶음,0,0,0,0,0,0,0,1,0,0
883,피자떡볶이,1,0,0,0,0,0,0,1,0,2
884,해물채소볶음,1,0,0,0,0,0,0,1,0,0


In [50]:
# CSR matrix를 사용하여 데이터를 변환
csr_data = csr_matrix(CFdata.drop('메뉴 이름', axis=1).values)

# cosine similarity 계산
cosine_sim = cosine_similarity(csr_data)

# 결과를 데이터프레임으로 변환
cosine_sim_df = pd.DataFrame(cosine_sim, index=CFdata['메뉴 이름'], columns=CFdata['메뉴 이름'])

# 유사도 행렬 확인
cosine_sim_df

메뉴 이름,견과류 영양솥밥,굴밥,김치볶음밥,꽃상추쌈,녹두죽,단팥죽,달래간장과 모둠버섯밥,닭날개주먹밥,닭안심죽,대구계란죽,...,김치계란피자,김치치킨도리아,대파 파프리카굴소스볶음,돈육불고기,두부케첩볶음,삶은무해산물볶음,엔다이브 위의 쇠고기볶음,피자떡볶이,해물채소볶음,햄마늘볶음
메뉴 이름,,,,,,,,,,,,,,,,,,,,,
견과류 영양솥밥,1.000000,1.000000,0.316228,0.707107,0.707107,0.707107,0.447214,1.000000,0.707107,0.707107,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
굴밥,1.000000,1.000000,0.316228,0.707107,0.707107,0.707107,0.447214,1.000000,0.707107,0.707107,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
김치볶음밥,0.316228,0.316228,1.000000,0.894427,0.223607,0.223607,0.989949,0.316228,0.223607,0.223607,...,0.900000,0.000000,0.000000,0.000000,0.000000,0.774597,0.000000,0.774597,0.000000,0.000000
꽃상추쌈,0.707107,0.707107,0.894427,1.000000,0.500000,0.500000,0.948683,0.707107,0.500000,0.500000,...,0.670820,0.000000,0.000000,0.000000,0.000000,0.577350,0.000000,0.577350,0.000000,0.000000
녹두죽,0.707107,0.707107,0.223607,0.500000,1.000000,1.000000,0.316228,0.707107,1.000000,1.000000,...,0.000000,0.000000,0.500000,0.500000,0.500000,0.288675,0.000000,0.288675,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
삶은무해산물볶음,0.000000,0.000000,0.774597,0.577350,0.288675,0.288675,0.730297,0.000000,0.288675,0.288675,...,0.903696,0.408248,0.577350,0.577350,0.577350,1.000000,0.408248,1.000000,0.577350,0.577350
엔다이브 위의 쇠고기볶음,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.316228,1.000000,0.707107,0.707107,0.707107,0.408248,1.000000,0.408248,0.707107,0.707107
피자떡볶이,0.000000,0.000000,0.774597,0.577350,0.288675,0.288675,0.730297,0.000000,0.288675,0.288675,...,0.903696,0.408248,0.577350,0.577350,0.577350,1.000000,0.408248,1.000000,0.577350,0.577350


In [53]:
cosine_sim_df.to_csv('cosine_sim.csv', encoding='utf-8-sig')